# ASX Share Price Viewer - Prototyping Notebook

_For MM app - colloboration between EGB and MJB._

## 1. Setup notebook

Note that on the Google Colab platform some Python libraries are pre-installed by default. Such libraries are pre-installed (in the Google Colab environment) and can just be imported.

In [1]:
import altair as alt
import pandas as pd
import requests
import json
from pathlib import Path

The `data_table` package is to enable a prettier and interactive display of Pandas dataframes.

The `files` module allows reading/writing files to the local file system (e.g. the laptop you're using).

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

from google.colab import files

Other libraries need to be installed, typically using the [`pip install`](https://pip.pypa.io/en/stable/getting-started/) command line tool. To run command line tools from within the (Jupyter) notebook environment you need to prefix the command with `!`. Note that these libraries will persist for the duration of the Google Colab session (i.e. if you re-run the installer it will just report that the package(s) is already installed.)

In [3]:
!pip install yahooquery

## 2. Get and verify data 

We need to establish:
* what data we need to solve the problem
* where we can source the data from
* what (if any) are the terms of use (for each data source)
* have we obtained the data correctly.

### What data do we need?

A list of companies in the [ASX 200](https://en.wikipedia.org/wiki/S%26P/ASX_200) index and their corresponding symbol (code/ticker); this includes their market capitalisation (i.e. how much they are worth) which defines which companies are in the index. While this can and does change through time, changes to the composition of the index are typically slow.

We also what market data and Yahoo Finance seeems to provide this information for free - someone has developed the `yahooquery` package that seems reasonable to use - but we should try and verify this!

#### ASX 200 - Index composition data

In [4]:
ASX_TICKERS_URL = "https://www.asx.com.au/asx/research/ASXListedCompanies.csv"

We can look at the first $n$ lines of the file using the `curl` and `head` commands in Unix (command line). Remember that the `!` character tells the notebook to execute the command "externally".

_To read about the "Unix command line" often also called the shell - see this [Intro to Bash](https://programminghistorian.org/en/lessons/intro-to-bash)._

In [5]:
!curl -s "$ASX_TICKERS_URL" | head -n 10

ASX listed companies as at Sat Jan 08 19:55:51 AEDT 2022

Company name,ASX code,GICS industry group
"MOQ LIMITED","MOQ","Software & Services"
"1414 DEGREES LIMITED","14D","Capital Goods"
"1ST GROUP LIMITED","1ST","Health Care Equipment & Services"
"29METALS LIMITED","29M","Materials"
"333D LIMITED","T3D","Commercial & Professional Services"
"360 CAPITAL ENHANCED INCOME FUND","TCF","Not Applic"
"360 CAPITAL GROUP","TGP","Real Estate"


We can see that this file has a header line i.e. "ASX listed companies as at..." so we need to skip this row.

In [6]:
asx_tickers_df = pd.read_csv(ASX_TICKERS_URL, skiprows=1)

Let's also look at the end (`tail`) of the file to see that it looks reasonable (i.e. is not obviously corrupt).

In [7]:
!curl -s "$ASX_TICKERS_URL" | tail -n 10

"ZETA RESOURCES LIMITED","ZER","Not Applic"
"ZEUS RESOURCES LIMITED","ZEU","Energy"
"ZICOM GROUP LIMITED","ZGL","Capital Goods"
"ZIMI LIMITED","ZMM","Technology Hardware & Equipment"
"ZIMPLATS HOLDINGS LIMITED","ZIM","Materials"
"ZINC OF IRELAND NL","ZMI","Materials"
"ZIP CO LIMITED.","Z1P","Diversified Financials"
"ZOOM2U TECHNOLOGIES LIMITED","Z2U","Transportation"
"ZOONO GROUP LIMITED","ZNO","Materials"
"ZULEIKA GOLD LIMITED","ZAG","Materials"


**EGB - TODO: Have a think about some reasonableness checks / tests for the data. i.e. to see that it makes good sense before we go using it and asking others to trust our application.** 

In [8]:
asx_tickers_df

,Company name,ASX code,GICS industry group
0,MOQ LIMITED,MOQ,Software & Services
1,1414 DEGREES LIMITED,14D,Capital Goods
2,1ST GROUP LIMITED,1ST,Health Care Equipment & Services
3,29METALS LIMITED,29M,Materials
4,333D LIMITED,T3D,Commercial & Professional Services
...,...,...,...
2260,ZINC OF IRELAND NL,ZMI,Materials
2261,ZIP CO LIMITED.,Z1P,Diversified Financials
2262,ZOOM2U TECHNOLOGIES LIMITED,Z2U,Transportation
2263,ZOONO GROUP LIMITED,ZNO,Materials


In [9]:
def lookup_company_name(asx_code):
  asx_code = asx_code.replace(".AX", "")
  return asx_tickers_df[asx_tickers_df["ASX code"] == asx_code]["Company name"].to_list()[0].title().replace(".", "")

In [10]:
company_name = lookup_company_name("CBA.AX")

In [11]:
company_name

'Commonwealth Bank Of Australia'

In [12]:
from yahooquery import Ticker

# Documentation: https://yahooquery.dpguthrie.com/guide/ticker/historical/

# See also: https://asxportfolio.com/shares-python-for-finance-getting-stock-data

# Example ASX API to get latest data -  e.g. CBA: https://www.asx.com.au/asx/1/share/CBA

In [13]:
ASX_DATA_URL = "https://www.asx.com.au/asx/1/share/"

In [14]:
def get_market_cap(asx_code):
  asx_code = asx_code.replace(".AX", "")
  r = requests.get(ASX_DATA_URL + asx_code)
  data = json.loads(r.text)
  try:
    return data["market_cap"], data["last_trade_date"]
  except:
    return None, None

In [15]:
get_market_cap("WBC.AX")[1]

'2022-01-07T00:00:00+1100'

In [16]:
USE_CACHED_DATA = False

In [ ]:
if USE_CACHED_DATA:
  if Path("asx_tickers_df.csv").exists():
    Path("asx_tickers_df.csv").unlink()   # delete file if it exists 
  uploaded = files.upload()
  filename = [fn for fn in uploaded.keys()]
  if filename[0] != "asx_tickers_df.csv":
    print("You need to upload asx_tickers_df.csv if you want to use cached data.")
  else:
    asx_tickers_df = pd.read_csv("asx_tickers_df.csv")
    asx_tickers_df.head()
    print("Loaded cached data file.")
else:
  asx_tickers_df["market_cap"] = asx_tickers_df["ASX code"].apply(lambda code: get_market_cap(code)[0])

Cache the tickers file with the market capitalisation information as it takes a few minutes to complete. We first save it to a temporary location on the Google Colab platform i.e. `/tmp` directory and then download it to our local file system.

In [ ]:
asx_tickers_df.to_csv("/tmp/asx_tickers_df.csv", index=False)

In [ ]:
files.download("/tmp/asx_tickers_df.csv")

## 3. Analysis / Organise data

What do we need to do / solve?

* Which companies are in the top 200 by market capitalisation?
* Allow user to choose from this list in a sensible manner e.g. lookup and/or predefined favourites list.
* Display the price history and other relevant information for each company selected in an appealing manner (to aide comparison of their relative performance). e.g. may want to look over past year, 6m, 3m, 1m, 1w etc.

#### Market data - Yahoo Finance

In [18]:
ticker_list = "CBA.AX WBC.AX"

In [19]:
start_date = "2021-01-01"
end_date = "2021-12-31"

In [20]:
tickers = Ticker(ticker_list)

In [21]:
tickers.key_stats

{'CBA.AX': {'52WeekChange': 0.20537806,
  'SandP52WeekChange': 0.23592949,
  'beta': 0.726338,
  'bookValue': 44.41,
  'category': None,
  'earningsQuarterlyGrowth': 0.546,
  'enterpriseToRevenue': 11.222,
  'enterpriseValue': 269011959808,
  'floatShares': 1767823060,
  'forwardEps': 5.17,
  'forwardPE': 19.854933,
  'fundFamily': None,
  'heldPercentInsiders': 0.00411,
  'heldPercentInstitutions': 0.19773,
  'lastDividendDate': 1629158400,
  'lastDividendValue': 2.0,
  'lastFiscalYearEnd': '2021-06-30 00:00:00',
  'lastSplitDate': '1999-10-06 00:00:00',
  'lastSplitFactor': '1:1',
  'legalType': None,
  'maxAge': 1,
  'mostRecentQuarter': '2021-06-30 00:00:00',
  'netIncomeToCommon': 8842999808,
  'nextFiscalYearEnd': '2023-06-30 00:00:00',
  'pegRatio': 4.31,
  'priceHint': 2,
  'priceToBook': 2.3114164,
  'profitMargins': 0.42470002,
  'sharesOutstanding': 1706390016,
  'trailingEps': 5.399},
 'WBC.AX': {'52WeekChange': 0.071956635,
  'SandP52WeekChange': 0.23592949,
  'beta': 0.84

In [22]:
tickers.summary_detail

{'CBA.AX': {'algorithm': None,
  'ask': 102.66,
  'askSize': 7100,
  'averageDailyVolume10Day': 1891981,
  'averageVolume': 2431212,
  'averageVolume10days': 1891981,
  'beta': 0.726338,
  'bid': 102.65,
  'bidSize': 36400,
  'currency': 'AUD',
  'dayHigh': 102.87,
  'dayLow': 101.89,
  'dividendRate': 4.0,
  'dividendYield': 0.04,
  'exDividendDate': '2021-08-17 00:00:00',
  'fiftyDayAverage': 100.7288,
  'fiftyTwoWeekHigh': 110.19,
  'fiftyTwoWeekLow': 81.56,
  'fiveYearAvgDividendYield': 5.04,
  'forwardPE': 19.854933,
  'fromCurrency': None,
  'lastMarket': None,
  'marketCap': 175160934400,
  'maxAge': 1,
  'open': 101.99,
  'payoutRatio': 0.527,
  'previousClose': 99.97,
  'priceHint': 2,
  'priceToSalesTrailing12Months': 7.306897,
  'regularMarketDayHigh': 102.87,
  'regularMarketDayLow': 101.89,
  'regularMarketOpen': 101.99,
  'regularMarketPreviousClose': 99.97,
  'regularMarketVolume': 2591847,
  'toCurrency': None,
  'tradeable': False,
  'trailingAnnualDividendRate': 3.5,


In [23]:
profile = tickers.asset_profile

In [24]:
profile['CBA.AX']['website']

'https://www.commbank.com.au'

In [25]:
profile['WBC.AX']['website']

'https://www.westpac.com.au'

In [26]:
#price_data_df = tickers.history(period='1y', interval='1d')

price_data_df = tickers.history(start=start_date, end=end_date)

In [27]:
price_data_df.index.names

FrozenList(['symbol', 'date'])

In [28]:
price_data_df.head(5)

high   volume  ...   adjclose  dividends
symbol date                            ...                      
CBA.AX 2021-01-04  83.919998  1414844  ...  80.689362        0.0
       2021-01-05  83.260002  1809541  ...  80.188377        0.0
       2021-01-06  83.349998  2252786  ...  79.870430        0.0
       2021-01-07  85.180000  3011086  ...  81.489044        0.0
       2021-01-08  85.629997  2446079  ...  82.500656        0.0

[5 rows x 7 columns]

In [29]:
price_data_df.describe()

,high,volume,close,low,open,adjclose,dividends
count,508.000000,5.080000e+02,508.000000,508.000000,508.000000,508.000000,508.000000
mean,60.560797,4.616489e+06,60.148799,59.712756,60.141319,59.141570,0.009213
std,36.570920,3.421135e+06,36.326418,36.037041,36.306376,36.124073,0.116687
min,19.500000,8.881680e+05,19.370001,19.230000,19.450001,18.444370,0.000000
25%,25.177500,2.195772e+06,25.020000,24.915000,25.030001,24.098135,0.000000
50%,54.835002,3.895154e+06,54.249999,54.139998,54.350000,53.056852,0.000000
75%,99.219997,5.962633e+06,98.627502,98.037498,98.705002,97.241383,0.000000
max,110.190002,3.626238e+07,110.129997,109.269997,110.000000,110.129997,2.000000


In [30]:
price_data_df.reset_index(inplace=True)

In [31]:
price_data_df.dtypes

symbol        object
date          object
high         float64
volume         int64
close        float64
low          float64
open         float64
adjclose     float64
dividends    float64
dtype: object

In [32]:
price_data_df.head()

,symbol,date,high,volume,close,low,open,adjclose,dividends
0,CBA.AX,2021-01-04,83.919998,1414844,83.750000,82.169998,82.230003,80.689362,0.0
1,CBA.AX,2021-01-05,83.260002,1809541,83.230003,82.485001,83.000000,80.188377,0.0
2,CBA.AX,2021-01-06,83.349998,2252786,82.900002,82.279999,82.690002,79.870430,0.0
3,CBA.AX,2021-01-07,85.180000,3011086,84.580002,83.699997,84.379997,81.489044,0.0
4,CBA.AX,2021-01-08,85.629997,2446079,85.629997,84.794998,84.989998,82.500656,0.0


In [33]:
len(price_data_df)

508

In [34]:
price_data_df['date'][0]

datetime.date(2021, 1, 4)

In [35]:
price_data_df["date"] = pd.to_datetime(price_data_df["date"])

In [36]:
chart_ticker = "WBC.AX"

In [37]:
chart_data_df = price_data_df[price_data_df["symbol"] == chart_ticker]

In [38]:
chart_data_df

,symbol,date,high,volume,close,low,open,adjclose,dividends
254,WBC.AX,2021-01-04,19.670000,4044153,19.629999,19.424999,19.459999,18.691944,0.0
255,WBC.AX,2021-01-05,19.530001,4944351,19.520000,19.379999,19.469999,18.587202,0.0
256,WBC.AX,2021-01-06,19.500000,7063517,19.370001,19.230000,19.450001,18.444370,0.0
257,WBC.AX,2021-01-07,20.340000,14348410,19.990000,19.780001,19.799999,19.034740,0.0
258,WBC.AX,2021-01-08,20.280001,7716248,20.280001,20.020000,20.110001,19.310884,0.0
...,...,...,...,...,...,...,...,...,...
503,WBC.AX,2021-12-23,21.180000,4359824,21.180000,21.040001,21.129999,21.180000,0.0
504,WBC.AX,2021-12-24,21.450001,3039364,21.200001,21.200001,21.260000,21.200001,0.0
505,WBC.AX,2021-12-29,21.650000,9676188,21.459999,21.400000,21.450001,21.459999,0.0
506,WBC.AX,2021-12-30,21.600000,4635337,21.500000,21.480000,21.510000,21.500000,0.0


In [39]:
c = alt.Chart(chart_data_df).mark_circle().encode(x='date', y='adjclose')

In [40]:
c

alt.Chart(...)